# 02 - Data from the Web

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.


## 1. World University Rankings by TopUniversities

---

We will focus on data scraping from the [TopUniversities 2018 rankings](https://www.topuniversities.com/university-rankings/world-university-rankings/2018), as well as asnwering the assignment questions by performing simple data analysis. The structure of this chapter follows the assignment outline:

1. Data scraping
2. Ranking: faculty members, students and internationalization
3. Ranking: regional and national results


### Libraries utilized

We will use the BeautifulSoup along with Pandas and standard Python utilities for performing the scraping and data analysis.

In [222]:
# importing the utils for Web scraping
import requests
from bs4 import BeautifulSoup

# importing python and data utils
import pandas as pd
import numpy as np
import json
%matplotlib inline
import matplotlib.pyplot as plt

### Swiss army knife for data scraping

[Postman](https://www.getpostman.com/) is utilized outside Python to get more sense about the Web jungle we are trying to harvest our data from. We have used both Interceptor and the application itself to make sense of incoming traffic. 

We have also relied on traffic observations in Chrome Development Tools (*which can be accessed in Chrome browsers on Windows platform by pressing F12*).

### 1.1 Data scraping

The first step in our data scraping process was to get acquainted with the website itself. We have turned on the Interceptor for the traffic and the application itself, then loaded the website and observed the requests. 

#### Making our life easier: Postman filters

It is useful to filter out results of requests sent to the Website of interest. In modern websites it is common to have many 3rd party APIs which would make it somewhat more difficult to find the requests of interest. Additionally, trafic from other tabs in browser (if open) will appear, so it is additionally useful to display only what is the focus of our search. Such benefits are visible in the practical example bellow:

<table width="70%">
  <tr>
    <th style="text-align:center">Requests without any filters</th>
    <th style="text-align:center">After filtering results from *www.topuniversities.com* only</th> 
  </tr>
  <tr>
    <td>![](images/postman0.png)</td>
    <td style="vertical-align:top">![](images/postman1.png)</td> 
  </tr>
</table>

From the vast list of various requests, we have localized what seems to be important for the given task, and the requests for resources for the TopUniversity rankings. By analyzing the requests, we can see one **GET** request which obtains a textual file:

[https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508597583828](https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508597583828)

By analyzing the file, it turns out to be a very useful JSON! All the data needed for scraping at this level is present here, most notably the ranking and the URL towards more information. It correlates to the same information we can directly see from the Chrome Developer Tools:

![](images/chrome0.png)

#####  Design choice
As in the image above can be seen, the HTML data is well organized in the tables, with appropriate classes which designate the content of the cell. One approach would be to use `BeautifulSoup` and parse the HTML of the page. Since we have the data in more concise format (JSON) we decided to process the textual representation. Since only the representation and intermediate steps differ, and results are the same, this is a choice we have made. 

Nevertheless, further steps will have to rely on parsing the HTML and DOM of the subsequent university links with further details (since no equivalent JSON is obtained), so both approaches are showcased.


#### Implementing the scraping procedure

We will extract the before found JSON file and use it as a baseline for scraping this Website. We send a GET request and process the file as JSON.

In [223]:
## TU = TopUniversities
TU_path = 'https://www.topuniversities.com'
base_json_path = '/sites/default/files/qs-rankings-data/357051.txt?_=1508597583828'

req = requests.get(TU_path+base_json_path)

TU_json = req.json()

We convert the JSON file to Pandas `DataFrame` to enable easier exploration and utilization of the file. We are interested in the **data** section of our JSON file, and we take the first 200 top ranking universities in the list. 

Notably, some universities share the same ranking (tie). Per assignment at this point we will take 200 best ranking universities for our analysis, and not the universities until the rank 200.

In [224]:
TU_df = pd.DataFrame.from_dict(TU_json['data']).head(200)

We can observe that we did not need any additional parsing which would add both to code complexity, potential computational overhead in bigger datasets, as well as algorithmic complexity if the website HTML code is not tidy (which is not the case here). The `DataFrame` carries all the information we need for now, of which the region, country, rank, title and url play the significant role in the task.

In [225]:
TU_df.head(2)

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,title,url
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,US,573,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",297282,2,North America,98.7,5,Stanford University,/universities/stanford-university


#### 1.1.1 Extracting the information for each University

Some information such as University name or rank is available immediately, while for detailed information we need to follow the subsequent link provided in the **url** field. Unfortunately, as previously mentioned, we don't have information packed in JSON format this time. We use HTML parsing provided by `BeautifulSoup` to obtain all the missing data. We are in luck since the HTML is well organized, with appropriate classes assigned to `div`s containing needed pieces of information.

We iterate through the previously obtained `DataFrame` and scrape and collect the necessary data, and append it to the existing `DataFrame`.

#### Note on undefined values

At least one of the universities has missing data and we need to handle such cases. If the number of such cases is small enough, we could impute the values manually, by consulting the appropriate sources, thus keeping the consistency of the rankings. If we don't have all the data for analysis, we would have to discard such Universities or come up with a methodology for mining up missing data on larger scale.

We choose to be consistent and we will print out the universities which have some missing values and will try to fill in the missing values.

In [226]:
# A helper function to get an number (integer) from the div text
def div2num(div):
    return int(div.find('div', class_='number').text.split()[0].replace(',',''))

In [227]:
for ind, row in TU_df.iterrows():
    # getting the detailed page of the given university
    # relative link to the page is given in the url field of the dataframe
    uni_details = requests.get(TU_path+row['url'])
    
    # get the HTML of the detailed university page and add it to the soup
    soup = BeautifulSoup(uni_details.text, 'html.parser')
    
    # get the name of the university from the DF
    uni_name = row['title']
    
    # get the number of faculty members from the detailed page and append to DF
    faculty_num = soup.find('div', class_='total faculty')
    try:
        # we parse the div text and append it to the DataFrame
        TU_df.loc[ind, 'Total faculty members'] = div2num(faculty_num)
    except:
        # print out the message in order to try to impute the data
        print('Undefined total faculty members for: '+uni_name)
    
    # similar procedure for other required data points
    students_num = soup.find('div', class_='total student')
    try:
        TU_df.loc[ind, 'Total students'] = div2num(students_num)
    except:
        print('Undefined total students for: '+uni_name)
    
    intl_faculty_num = soup.find('div', class_='inter faculty')
    try:
        TU_df.loc[ind, 'International faculty members'] = div2num(intl_faculty_num)
    except:
        print('Undefined international faculty members for: '+uni_name)
    
    intl_students_num = soup.find('div', class_='total inter')
    try:
        TU_df.loc[ind, 'International students'] = div2num(intl_students_num)
    except:
        print('Undefined international students for: '+uni_name)

Undefined total faculty members for: New York University (NYU)
Undefined total students for: New York University (NYU)
Undefined international faculty members for: New York University (NYU)
Undefined international students for: New York University (NYU)
Undefined international faculty members for: Indian Institute of Science (IISc) Bangalore


#### Imputing undefined values

We can see that the New York University (NYU) has information that is completely missing, while Indian Institute of Science (IISc) Bangalore has one missing value. Considering the scope of missing values, they will be manually set using the following sources:

- NYU: https://www.nyu.edu/about/news-publications/nyu-at-a-glance.html, https://www.nyu.edu/admissions/undergraduate-admissions/nyu-facts.html
- IISc: http://www.iisc.ac.in/iisc-in-numbers/#ffs-tabbed-12

This is potentially troubling, since there is no information about the actual data collection for the ranking, methodology or actual date which is available for a confident selection of the figures missing. Frequently, the websites also omit the precise dates. Comparing the potential noise which would be added to the fact that we would discard 2 top 200 universities, the decision is to take the data and impute it, while being aware that it might induce some error in methodology. It is a tradeoff between statistical error in imputed data and the error which would occur in the further aggregate analysis.

Therefore the NYU data is following:
- total faculty members: 7861 (Fall 2015)
- total students: 58419 (Fall 2015)
- international faculty members: 619 (September 2017)
- international students: 26% == 15189 (Fall 2015)

Unfortunately, the data of international faculty members is not available in public records online for IISc, therefore it will be designated as 0. The unavailability might rise from the fact that the figure is rather low (https://scroll.in/article/811696/there-are-no-indian-universities-in-the-worlds-top-250-list-heres-how-javadekar-can-change-that), so it would not be positive for faculty to publicize such information. 

*If time permitted a good practice would be to contact the institution directly for the actual quote in both cases*

In [228]:
TU_df.loc[TU_df.title=='New York University (NYU)','Total faculty members'] = 7861
TU_df.loc[TU_df.title=='New York University (NYU)','Total students'] = 58419
TU_df.loc[TU_df.title=='New York University (NYU)','International faculty members'] = 619
TU_df.loc[TU_df.title=='New York University (NYU)','International students'] = 15189


TU_df = TU_df.fillna(0)

#### Pruning the data in DataFrame

We have several pieces of information that are redundant at this point, so we will remove such columns from the DataFrame for a better overview. We can see bellow that *cc*, *rank_display*, *stars*, *core_id*, *guide*, *logo*, *nid* and *url* are present, but redundant in the following analysis.

In [229]:
TU_df.head(1)

,cc,core_id,country,guide,logo,nid,rank_display,region,score,stars,title,url,Total faculty members,Total students,International faculty members,International students
0,US,410,United States,"<a href=""/where-to-study/north-america/united-...","<img src=""https://www.topuniversities.com/site...",294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...,2982.0,11067.0,1679.0,3717.0


In [230]:
TU_df.drop(['cc', 'stars', 'core_id', 'guide', 'logo', 'nid', 'url'], axis=1, inplace=True)
TU_df.head(1)

,country,rank_display,region,score,title,Total faculty members,Total students,International faculty members,International students
0,United States,1,North America,100,Massachusetts Institute of Technology (MIT),2982.0,11067.0,1679.0,3717.0


#### We have our DataFrame ready for further analysis!
---

### 1.2 Ranking: faculty members, students and internationalization

In this section we answer the question: **Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?**

For easier analysis for (a) and (b) we will add the ratios to the existing DataFrame:

In [19]:
# ratio between faculty members and students
TU_df['Ratio faculty/students'] = TU_df['Total faculty members']/TU_df['Total students']

# ratio of international students
TU_df['Ratio intl/total students'] = TU_df['International students']/TU_df['Total students']

In [20]:
TU_df.head(1)

,country,rank_display,region,score,title,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
0,United States,1,North America,100,Massachusetts Institute of Technology (MIT),2982.0,11067.0,1679.0,3717.0,0.26945,0.335863


We have prepared the data for our analysis! The results are obtained by sorting the `DataFrame` by appropriate criterion.

### Best university: (a) ratio between faculty members and students

In [21]:
TU_fac_stud_df = TU_df.sort_values('Ratio faculty/students', ascending=False)

#### Top 5 by ratio of faculty members and students:

In [22]:
TU_fac_stud_df.head(5)

,country,rank_display,region,score,title,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
3,United States,4,North America,97.7,California Institute of Technology (Caltech),953.0,2255.0,350.0,647.0,0.422616,0.286918
15,United States,16,North America,90.4,Yale University,4940.0,12402.0,1708.0,2469.0,0.398323,0.199081
5,United Kingdom,6,Europe,95.3,University of Oxford,6750.0,19720.0,2964.0,7353.0,0.342292,0.372870
4,United Kingdom,5,Europe,95.6,University of Cambridge,5490.0,18770.0,2278.0,6699.0,0.292488,0.356899
16,United States,17,North America,89.8,Johns Hopkins University,4462.0,16146.0,1061.0,4105.0,0.276353,0.254243


### Best university: (b) ratio of international students

In [23]:
TU_intl_df = TU_df.sort_values('Ratio intl/total students', ascending=False)

#### Top 5 by ratio of international students

In [24]:
TU_intl_df.head(5)

,country,rank_display,region,score,title,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
34,United Kingdom,35,Europe,81.8,London School of Economics and Political Scien...,1088.0,9760.0,687.0,6748.0,0.111475,0.691393
11,Switzerland,12,Europe,91.2,Ecole Polytechnique Fédérale de Lausanne (EPFL),1695.0,10343.0,1300.0,5896.0,0.163879,0.570047
7,United Kingdom,8,Europe,93.7,Imperial College London,3930.0,16090.0,2071.0,8746.0,0.244251,0.543567
198,Netherlands,200,Europe,47.9,Maastricht University,1277.0,16385.0,502.0,8234.0,0.077937,0.502533
47,United States,=47,North America,78.6,Carnegie Mellon University,1342.0,13356.0,425.0,6385.0,0.100479,0.478062


------
### 1.3 Ranking: regional and national results

In this section we use the results obtained in **1.2** and aggregate them, separately, by country and region, thus providing answer to the question 2:

*Answer the previous question aggregating the data by (c) country and (d) region.*

The `DataFrame` contains all the datapoints we need for answering the questions, the only procedure we need to undertake is aggregation of data. We need to remove the calculated ratios per faculty since the summation would not result in proper result in this case (we would need to normalize the result otherwise). We can further remove the score and name of the university **manually**, since it does not play a role in the dataset, but since it is a variable on which we could not perform aggregation such variables would anyway be ignored by *Pandas*.

In [25]:
TU_aggregate_df = TU_df.copy()
TU_aggregate_df.drop(['Ratio faculty/students', 'Ratio intl/total students'], axis=1, inplace=True)
TU_aggregate_df.head(1)

,country,rank_display,region,score,title,Total faculty members,Total students,International faculty members,International students
0,United States,1,North America,100,Massachusetts Institute of Technology (MIT),2982.0,11067.0,1679.0,3717.0


---
#### Aggregating data by country (c)

We will group the data by the country and then sum the values of such aggregate to produce desired data.

In [26]:
# we can compose multiple functions to reduce code clutter
TU_country_df = TU_aggregate_df.groupby(['country']).sum()
TU_country_df.head(2)

,Total faculty members,Total students,International faculty members,International students
country,,,,
Argentina,16421.0,122301.0,3165.0,27109.0
Australia,22034.0,301994.0,11382.0,106359.0


As done previously, desired ratios are calculated and added to the `DataFrame`:

In [27]:
TU_country_df['Ratio faculty/students'] = TU_country_df['Total faculty members']/TU_country_df['Total students']
TU_country_df['Ratio intl/total students'] = TU_country_df['International students']/TU_country_df['Total students']

#### Top 5 countries with Universities with highest ratio of faculty to students

In [28]:
TU_country_df.sort_values('Ratio faculty/students', ascending=False).head(5)

,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
country,,,,,,
Russia,6709.0,30233.0,373.0,5098.0,0.221910,0.168624
Denmark,11916.0,67223.0,3904.0,9543.0,0.177261,0.141960
Saudi Arabia,1062.0,6040.0,665.0,989.0,0.175828,0.163742
Singapore,9444.0,58466.0,6079.0,16168.0,0.161530,0.276537
Malaysia,2755.0,17902.0,655.0,3476.0,0.153893,0.194168


#### Top 5 countries with Universities with highest ratio of international students

In [29]:
TU_country_df.sort_values('Ratio intl/total students', ascending=False).head(5)

,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
country,,,,,,
Australia,22034.0,301994.0,11382.0,106359.0,0.072962,0.352189
United Kingdom,79934.0,583621.0,30216.0,199426.0,0.136962,0.341705
Hong Kong,10166.0,78838.0,6296.0,24499.0,0.128948,0.310751
Austria,4117.0,63446.0,1572.0,19667.0,0.064890,0.309980
Switzerland,15323.0,109112.0,9208.0,32995.0,0.140434,0.302396


---
#### Aggregating data by region (d)

We perform the same procedure with grouping by the region instead of country.

In [30]:
TU_region_df = TU_aggregate_df.groupby(['region']).sum()
TU_region_df.head(2)

,Total faculty members,Total students,International faculty members,International students
region,,,,
Africa,1733.0,19593.0,379.0,3325.0
Asia,106734.0,807003.0,25462.0,110100.0


We calculate the desired ratios by the region and then present the top 5 regions by each criterion.

In [31]:
TU_region_df['Ratio faculty/students'] = TU_region_df['Total faculty members']/TU_region_df['Total students']
TU_region_df['Ratio intl/total students'] = TU_region_df['International students']/TU_region_df['Total students']

#### Top 5 regions with Universities with the highest faculty to student ratio

In [32]:
TU_region_df.sort_values(['Ratio faculty/students'], ascending=False).head(5)

,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
region,,,,,,
Asia,106734.0,807003.0,25462.0,110100.0,0.132260,0.136431
North America,189984.0,1604772.0,44455.0,307305.0,0.118387,0.191494
Europe,218358.0,1957251.0,67598.0,449364.0,0.111564,0.229589
Latin America,45382.0,435750.0,5648.0,36871.0,0.104147,0.084615
Africa,1733.0,19593.0,379.0,3325.0,0.088450,0.169703


#### Top 5 regions with Universities with the highest ratio of international students

In [33]:
TU_region_df.sort_values(['Ratio intl/total students'], ascending=False).head(5)

,Total faculty members,Total students,International faculty members,International students,Ratio faculty/students,Ratio intl/total students
region,,,,,,
Oceania,25347.0,350167.0,12786.0,118798.0,0.072385,0.339261
Europe,218358.0,1957251.0,67598.0,449364.0,0.111564,0.229589
North America,189984.0,1604772.0,44455.0,307305.0,0.118387,0.191494
Africa,1733.0,19593.0,379.0,3325.0,0.088450,0.169703
Asia,106734.0,807003.0,25462.0,110100.0,0.132260,0.136431


## 2. World University Rankings by TimesHigherEducation

In this chapter, we will scrape the data from [TimesHigherEducation ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking), as well as answering the assignment questions by performing simple data analysis.


Pandas options setting for a better view of the data

In [34]:
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

### Analysis of the website to scrape

As in the case for task 1, we notice, using Interceptor Chrome app and Postman, that when accessing the main list url

http://timeshighereducation.com/world-university-rankings/2018/world-ranking 


another specific **GET** request is made to one of the server's REST API endpoints

https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json

which serves a file in JSON format that contains a data field with information about each university in the ranking.

#### Implementing the scrapping procedure

We begin by extracting the JSON file found previously with Interceptor and Postman and processing it as a JSON. In order to do so, we implement an utility function that sends a **GET** request to a server's API endpoint identified by the url.

The result is saved in a dictionary so that the http request is made only once per url.

In [35]:
json_data = {}
def get_json(url):
    if url not in json_data:
        r = requests.get(url)
        json_data[url] = r
    else:
        r = json_data[url]
        
    return r.json()

In [36]:
thedu_url = 'https://www.timeshighereducation.com'
thedu_json_url = '/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'

json_data_thedu = get_json(thedu_url + thedu_json_url)

#### Analysis of the json object format

In [37]:
# fields of the entire json object
display(json_data_thedu.keys())
# one entry in the data field
display(json_data_thedu['data'][0])

dict_keys(['pillars', 'data', 'locations', 'subjects'])

{'aliases': 'University of Oxford',
 'location': 'United Kingdom',
 'member_level': '0',
 'name': 'University of Oxford',
 'nid': 468,
 'rank': '1',
 'rank_order': '10',
 'record_type': 'master_account',
 'scores_citations': '99.1',
 'scores_citations_rank': '15',
 'scores_industry_income': '63.7',
 'scores_industry_income_rank': '169',
 'scores_international_outlook': '95.0',
 'scores_international_outlook_rank': '24',
 'scores_overall': '94.3',
 'scores_overall_rank': '10',
 'scores_research': '99.5',
 'scores_research_rank': '1',
 'scores_teaching': '86.7',
 'scores_teaching_rank': '5',
 'stats_female_male_ratio': '46 : 54',
 'stats_number_students': '20,409',
 'stats_pc_intl_students': '38%',
 'stats_student_staff_ratio': '11.2',
 'subjects_offered': 'Archaeology,Art, Performing Arts & Design,Biological Sciences,Business & Management,Chemical Engineering,Chemistry,Civil Engineering,Computer Science,Economics & Econometrics,Electrical & Electronic Engineering,General Engineering,Geo

The retrieved json object has 4 fields: data, subjects, locations and pillars out of which we are interested only in data. Specifically in the first 200 entries in the corresponding array because, again, we are focusing on the 200 best ranking universities and not the universities until rank 200.

#### Structure of data

We observe that the data field contains the name, location which is basically the country, number of students, ratio of international students and ratio of student to staff which is enough to infer all the fields that are asked in the asignment, except the region and number of international faculty members. Therefore we are not required to aditionally scrape the detailed page of the univerities.

Given that the region of a country cannot variate, for this field we will use external information, for example the dataframe from the previous task, to infer the region of an university. 

The number of international faculty members is not provided by the TimesHigherEducation, but we do not need it in our further analysis related to this task.

Observing the data field is a list of records, it makes sense to load it in a DataFrame for easier manipulation.

In [38]:
data_thedu = pd.DataFrame(json_data_thedu['data'])

In [39]:
data_thedu.head()

,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,scores_industry_income,scores_industry_income_rank,scores_international_outlook,scores_international_outlook_rank,scores_overall,scores_overall_rank,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,63.7,169,95.0,24,94.3,10,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,51.5,260,93.0,35,93.2,20,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,92.6,51,59.7,322,93.0,30,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,60.5,189,77.6,162,93.0,40,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,88.4,63,87.6,81,92.5,50,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...


In [40]:
data_thedu.columns

Index(['aliases', 'location', 'member_level', 'name', 'nid', 'rank',
       'rank_order', 'record_type', 'scores_citations',
       'scores_citations_rank', 'scores_industry_income',
       'scores_industry_income_rank', 'scores_international_outlook',
       'scores_international_outlook_rank', 'scores_overall',
       'scores_overall_rank', 'scores_research', 'scores_research_rank',
       'scores_teaching', 'scores_teaching_rank', 'stats_female_male_ratio',
       'stats_number_students', 'stats_pc_intl_students',
       'stats_student_staff_ratio', 'subjects_offered', 'url'],
      dtype='object')

#### Restructuring the data

As argued above, we will keep in the DataFrame only the columns that are relevant to our analysis:
- name
- rank
- location
- stats_number_students
- stats_pc_intl_students
- stats_student_staff_ratio

In [41]:
columns_to_drop = data_thedu.columns.difference(['name', 'rank','location', 'stats_number_students', 'stats_pc_intl_students', 'stats_student_staff_ratio'])
data_thedu = data_thedu.drop(columns_to_drop, axis=1)[0:200]

In [42]:
data_thedu.stats_pc_intl_students.head()

0    38%
1    35%
2    27%
3    22%
4    34%
Name: stats_pc_intl_students, dtype: object

The column stats_pc_intl_students representing the percentage of international students does not have a format that allows us do the operations necessary to infer the number of international students. 

We will create a function that will parse the string values of this column and transform them into a value from [0,1]

In [43]:
def percentage_to_ratio(value):
    # we slice the value up until the last character
    # then parse it as an int and divide it by 100 to obtain a value between 0 and 1
    return int(value[:-1])/100

In [44]:
data_thedu['stats_pc_intl_students'] = data_thedu['stats_pc_intl_students'].apply(percentage_to_ratio)
data_thedu.rename(columns={'stats_pc_intl_students':'stats_intl_students_ratio'}, inplace=True)

In [45]:
data_thedu.head()

,location,name,rank,stats_number_students,stats_intl_students_ratio,stats_student_staff_ratio
0,United Kingdom,University of Oxford,1,"20,409",0.38,11.2
1,United Kingdom,University of Cambridge,2,"18,389",0.35,10.9
2,United States,California Institute of Technology,=3,"2,209",0.27,6.5
3,United States,Stanford University,=3,"15,845",0.22,7.5
4,United States,Massachusetts Institute of Technology,5,"11,177",0.34,8.7


### Inferring data

We can now proceed to infer the number of international students and the number of faculty members

The number of international students can be calculated as the product between the number of students and the international students ratio

The number of faculty members can be calculated as the number of students divided by the student per staff ratio

#### Data types

In [46]:
data_thedu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
location                     200 non-null object
name                         200 non-null object
rank                         200 non-null object
stats_number_students        200 non-null object
stats_intl_students_ratio    200 non-null float64
stats_student_staff_ratio    200 non-null object
dtypes: float64(1), object(5)
memory usage: 9.5+ KB


As we can see, all the stats, except the transformed international students ratio, have the data type object.

For us to able to infer the new columns values, we will have to parse stats_number_students to int and stats_student_staff_ratio to float.

In [47]:
data_thedu['stats_number_students'] = data_thedu.stats_number_students.apply(lambda x: int(x.replace(',','')))
data_thedu['stats_student_staff_ratio'] = data_thedu.stats_student_staff_ratio.apply(lambda x: float(x))

We now compute the new columns as described above

In [48]:
data_thedu['number_international_students_inferred'] = data_thedu.stats_number_students * data_thedu.stats_intl_students_ratio
data_thedu['number_faculty_members_inferred'] = data_thedu.stats_number_students / data_thedu.stats_student_staff_ratio

Having only two decimals precision for the ratio, the resulting values for the inferred columns are not all integers but we choose to leave them as that in order to maintain the accuracy of our data.

One thing left to do is to transform the student per staff ratio into staff per student ratio as per assignment.

In [49]:
data_thedu['stats_student_staff_ratio'] = 1 / data_thedu.stats_student_staff_ratio
data_thedu.rename(columns={'stats_student_staff_ratio':'stats_staff_student_ratio'}, inplace=True)

In [50]:
data_thedu.head()

,location,name,rank,stats_number_students,stats_intl_students_ratio,stats_staff_student_ratio,number_international_students_inferred,number_faculty_members_inferred
0,United Kingdom,University of Oxford,1,20409,0.38,0.089286,7755.42,1822.232143
1,United Kingdom,University of Cambridge,2,18389,0.35,0.091743,6436.15,1687.064220
2,United States,California Institute of Technology,=3,2209,0.27,0.153846,596.43,339.846154
3,United States,Stanford University,=3,15845,0.22,0.133333,3485.90,2112.666667
4,United States,Massachusetts Institute of Technology,5,11177,0.34,0.114943,3800.18,1284.712644


We have prepared the data for our analysis! The results are obtained by sorting the `DataFrame` by appropriate criterion.

### Best university: (a) ratio between faculty members and students

In [51]:
thedu_fac_stud_df = data_thedu.sort_values('stats_staff_student_ratio', ascending=False)

#### Top 5 by ratio of faculty members and students

In [52]:
thedu_fac_stud_df.head()

,location,name,rank,stats_number_students,stats_intl_students_ratio,stats_staff_student_ratio,number_international_students_inferred,number_faculty_members_inferred
105,United States,Vanderbilt University,=105,12011,0.13,0.303030,1561.43,3639.696970
109,Denmark,University of Copenhagen,=109,30395,0.14,0.243902,4255.30,7413.414634
153,United States,University of Rochester,=153,9636,0.29,0.232558,2794.44,2240.930233
11,United States,Yale University,12,12155,0.21,0.232558,2552.55,2826.744186
12,United States,Johns Hopkins University,13,15498,0.24,0.232558,3719.52,3604.186047


### Best university: (b) ratio of international students

In [53]:
thedu_intl = data_thedu.sort_values('stats_intl_students_ratio', ascending=False)

#### Top 5 by ratio of international students

In [54]:
thedu_intl.head()

,location,name,rank,stats_number_students,stats_intl_students_ratio,stats_staff_student_ratio,number_international_students_inferred,number_faculty_members_inferred
24,United Kingdom,London School of Economics and Political Science,=25,10065,0.71,0.081967,7146.15,825.000000
178,Luxembourg,University of Luxembourg,=179,4969,0.57,0.068493,2832.33,340.342466
37,Switzerland,École Polytechnique Fédérale de Lausanne,=38,9928,0.55,0.089286,5460.40,886.428571
7,United Kingdom,Imperial College London,8,15857,0.55,0.087719,8721.35,1390.964912
102,Netherlands,Maastricht University,103,16727,0.50,0.055556,8363.50,929.277778


## 3. Merging the two DataFrames

Task: Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

Solution: for this task we will try to normalize the universities name in both datasets, i.e. try to extract only the useful information from each university name and then try to match only using that information. For that, we first need to understand better our data. As we observed, we have multiple universities names followed by their acronyms in the TU dataset. Let's analyze in a more in-dept manner those cases:

In [231]:
TU_df[TU_df.title.apply(lambda x: True if '(' in x else False)].title

0            Massachusetts Institute of Technology (MIT)
3           California Institute of Technology (Caltech)
6                        UCL (University College London)
10     Nanyang Technological University, Singapore (NTU)
11       Ecole Polytechnique Fédérale de Lausanne (EPFL)
14                National University of Singapore (NUS)
26              University of California, Berkeley (UCB)
32          University of California, Los Angeles (UCLA)
34     London School of Economics and Political Scien...
38            University of California, San Diego (UCSD)
44       The University of New South Wales (UNSW Sydney)
45            The Chinese University of Hong Kong (CUHK)
51                             New York University (NYU)
70     Pohang University of Science And Technology (P...
74                     Universidad de Buenos Aires (UBA)
76                      National Taiwan University (NTU)
107                       Sungkyunkwan University (SKKU)
113                            

It can be seen that, besides one name, all the ones that contain a parantheses keep the acronym of the university appended to the name. It is not the case of UCL, where in fact the full name is in parantheses, while the acronym is outside. We will have to handle this case.

Also, one really important thing to notice is that some universities have their name stated in the native language in one ranking, while in the other one is in English. This is the case, for example, of **Technical University of Berlin**, as named in the THE tanking, which in is named **Technische Universität Berlin (TU Berlin)** in the TU ranking. Therefore, one more thing we should handle should be language differences. The solution for that would be to translate every university name in English language, using an API which makes requests to Google Translate servers. In that manner, all names will be expressed in the same language.

Another issue is that some names have some additional not-so-meaningful words in their structure, as *of* or *the*. An example of this is **The university of Hong Kong**, as per TU ranking, and **University of Hong Kong**, as expressed in THE ranking. This problem will be solved by eliminating the unncecessary words and punctuation signs.

Last issue concerned is the words order. We might have universities that appear in both rankings, but with different order of words, or even the different order might be injected by the previous translation. We solve the problem by representing the normalized name as a set of words, so that the order of the words don't matter when similarity is established.

Therefore, the solution for normalizing names is as follows: for **UCL**, transform the name into **University College London**, to keep consistency in the dataset. Then, get rid of the acronym when exists, transform name to lower case and replace punctation marks in name. After that, translate name in English, get rid of the linking words and then split the name into words. Finally, the representation will be consisting of the set of all words resulted from the last operation. The implementation is presented below:

In [147]:
from googletrans import Translator
# googletrans is a library which makes calls to the google translate servers in order to translate words or sentences.

def normalize_university_name(name):
    translator = Translator()
    if name == 'UCL (University College London)':
        name = 'University College London' 
        # UCL is the only university for which the acronym is first and full name in parantheses, solve that problem by hand
    name = name.lower()
    CHARACTERS_REPLACE_BEFORE_TRANSLATE = [' – ', ' - ', '-', ' & ', ', '] # characters which don't contain letters
    CHARACTERS_REPLACE_AFTER_TRANSLATE = [' and ', ' in ', ' of ', ' from '] # english words (used after translation)
    
    name = name.replace('’', "'") # replace weird character
    for character_to_replace in CHARACTERS_REPLACE_BEFORE_TRANSLATE:
        name = name.replace(character_to_replace, ' ') 
    
    if '(' in name:
        name = name.rpartition('(')[0] # consider only the string before the last occurence of (, which is the full name of the university
        
    name = translator.translate(name, dest='en').text # transform the name of the university into English text, so that we can compare words
    
    for character_to_replace in CHARACTERS_REPLACE_AFTER_TRANSLATE:
        name = name.replace(character_to_replace, ' ') 
    
    name = name.lower() # needed again, Google translate might be a "good" hero and transform everything in big letters 
    words = name.strip().split() # split the university name in words
    return set(words) # return the words representing university name as a set, for unordered checking   


After we found a good way of normalizing the names, we must use it for establishing the link between universities using the names in the two dataset. For that, we will consider, for each university in the TU dataset, all of the universities from THE dataset that are from the same country (FIRST thing to check should be country, if not from the same country then clearly not same university) and have the property that all the words in one representation are present in the other representation. For example, we will want that **Scuola Superiore Sant'Anna Pisa di Studi Universitari e di Perfezionamento** is matched by **Scuola Superiore Sant’Anna**, which is possible by using this condition. Furthermore, from all the matching universities, we will select the one that has the most common words with the current university. 

In [233]:
def delete_entry_by_university_name(university_name, universities_normalized):
    for index, pair in enumerate(universities_normalized):
        if pair[1] == university_name:
            del universities_normalized[index]
            return
        
def pair_universities():
    THE_universities_normalized = []
    for university_data in data_thedu[['name', 'location']].itertuples():
        THE_universities_normalized.append((normalize_university_name(university_data.name), university_data.name, university_data.location))
        # keep tuples of the form (normalized_name, name, location)

    paired = []
    unpaired = []

    for university_data in TU_df[['title', 'country']].itertuples():
        normalized_university = normalize_university_name(university_data.title)
        country_university = university_data.country

        best_words_match_number = -1 # initially, we set the biggest number of matches to -1
        best_match_university = None # initially, the best match university is None

        for THE_normalized_university, THE_university, THE_country in THE_universities_normalized:
            if THE_country.lower() != country_university.lower():
                continue # if not from the same country, then it's not a match

            if normalized_university.issubset(THE_normalized_university) or THE_normalized_university.issubset(normalized_university):
                words_intersection = normalized_university.intersection(THE_normalized_university)

                if len(words_intersection) > best_words_match_number:
                    best_words_match_number = len(words_intersection)
                    best_match_university = THE_university

        if best_match_university is not None: # we found match
            paired.append((university_data.title, best_match_university))
            delete_entry_by_university_name(best_match_university, THE_universities_normalized) # delete matched university from the list

        else:
            unpaired.append((university_data.title, university_data.country))

    THE_unpaired = [(pair[1], pair[2]) for pair in THE_universities_normalized]

    return paired, unpaired, THE_unpaired


In [234]:
paired, TU_unpaired, THE_unpaired = pair_universities() # Code for pairing universities

Next, we want to make sure we did not insert some inconsistencies. We will print every pair and will actually inspect it for mistakes. The reviewer should understand that printing all the values is a necessity in this case:

In [201]:
TU_pair = pd.Series([element[0] for element in paired])
THE_pair = pd.Series([element[1] for element in paired])

paired_dataframe = pd.DataFrame({'TU' : TU_pair, 'THE' : THE_pair})
paired_dataframe

,THE,TU
0,Massachusetts Institute of Technology,Massachusetts Institute of Technology (MIT)
1,Stanford University,Stanford University
2,Harvard University,Harvard University
3,California Institute of Technology,California Institute of Technology (Caltech)
4,University of Cambridge,University of Cambridge
5,University of Oxford,University of Oxford
6,University College London,University College London
7,Imperial College London,Imperial College London
8,University of Chicago,University of Chicago
9,ETH Zurich – Swiss Federal Institute of Techno...,ETH Zurich - Swiss Federal Institute of Techno...


Our analysis lead to discovering two bad pairings: (***Universitat de Barcelona***, ***Autonomous University of Barcelona***) and ***Vrije Universiteit Brussel (VUB)***, ***Université Libre de Bruxelles***). We will have to delete these two pairs from the list. Now, let's inspect the unpaired university names, indexed by country, as below:

In [236]:
TU_unpaired_name = pd.Series([element[0] for element in TU_unpaired])
TU_unpaired_country = pd.Series([element[1] for element in TU_unpaired])

TU_unpaired_dataframe = pd.DataFrame({'name' : TU_unpaired_name, 'country' : TU_unpaired_country})
TU_unpaired_dataframe['source'] = 'TU'

TU_unpaired_dataframe = TU_unpaired_dataframe.set_index(['country', 'source']).sort_index()

TU_unpaired_dataframe.head()

name
country   source                                             
Argentina TU                Universidad de Buenos Aires (UBA)
Australia TU                  University of Technology Sydney
Austria   TU                  Vienna University of Technology
Brazil    TU                        Universidade de São Paulo
          TU      Universidade Estadual de Campinas (Unicamp)

Now for THE dataset, the unpaired universities are:

In [237]:
THE_unpaired_name = pd.Series([element[0] for element in THE_unpaired])
THE_unpaired_country = pd.Series([element[1] for element in THE_unpaired])

THE_unpaired_dataframe = pd.DataFrame({'name' : THE_unpaired_name, 'country' : THE_unpaired_country})
THE_unpaired_dataframe['source'] = 'THE'

THE_unpaired_dataframe = THE_unpaired_dataframe.set_index(['country', 'source']).sort_index()

THE_unpaired_dataframe.head()

name
country source                                                   
France  THE     Paris Sciences et Lettres – PSL Research Unive...
        THE                                  Paris-Sud University
        THE                   Paris-Sorbonne University – Paris 4
Germany THE                                            LMU Munich
        THE                                University of Freiburg

Finally, we will merge the two datasets and index it by country. Afterwards, we can look for the universities in the same country, for pairs between THE and TU. Let's take a look:

In [238]:
merged_unpaired_dataframe = pd.concat([TU_unpaired_dataframe, THE_unpaired_dataframe]).sort_index()
merged_unpaired_dataframe

name
country            source                                                   
Argentina          TU                      Universidad de Buenos Aires (UBA)
Australia          TU                        University of Technology Sydney
Austria            TU                        Vienna University of Technology
Brazil             TU                              Universidade de São Paulo
                   TU            Universidade Estadual de Campinas (Unicamp)
Canada             TU                                 University of Waterloo
Chile              TU          Pontificia Universidad Católica de Chile (UC)
                   TU                                   Universidad de Chile
France             THE     Paris Sciences et Lettres – PSL Research Unive...
                   THE                                  Paris-Sud University
                   THE                   Paris-Sorbonne University – Paris 4
                   TU                        Ecole normale supérieure, Paris
                   TU                                        CentraleSupélec
Germany            THE                                            LMU Munich
                   THE                                University of Freiburg
                   THE                                    University of Bonn
                   THE                                University of Mannheim
                   THE                  Charité - Universitätsmedizin Berlin
                   THE                                 University of Cologne
                   THE                                            TU Dresden
                   THE                                        Ulm University
                   THE                      University of Erlangen-Nuremberg
                   THE                                University of Würzburg
                   THE                                 University of Münster
                   TU                 Ludwig-Maximilians-Universität München
                   TU                   Albert-Ludwigs-Universitaet Freiburg
India              TU            Indian Institute of Technology Delhi (IITD)
                   TU           Indian Institute of Technology Bombay (IITB)
                   TU           Indian Institute of Science (IISc) Bangalore
Ireland            TU                              University College Dublin
Israel             TU                     The Hebrew University of Jerusalem
Italy              TU                                  Politecnico di Milano
                   TU           Alma Mater Studiorum - University of Bologna
Japan              TU                          Tokyo Institute of Technology
                   TU                                       Osaka University
                   TU                                      Tohoku University
                   TU                                      Nagoya University
                   TU                                    Hokkaido University
                   TU                                      Kyushu University
                   TU                               Keio University CEMS MIM
Luxembourg         THE                              University of Luxembourg
Malaysia           TU                                 Universiti Malaya (UM)
Mexico             TU        Universidad Nacional Autónoma de México  (UNAM)
                   TU      Instituto Tecnológico y de Estudios Superiores...
Netherlands        THE                           Radboud University Nijmegen
                   THE                          Vrije Universiteit Amsterdam
                   THE                                    Tilburg University
New Zealand        TU                                    University of Otago
Norway             TU                                   University of Bergen
Russia             TU                      Lomonosov Moscow State University
Russian Federation THE                     Lomonosov Moscow State Un

We can see that most of the universities are unrelated, but some of them were not detected as matching, even though they are in reality. Several are really hard to detect even by humans, without some extra information that could be found by using Google. A good example for that is ***Albert-Ludwigs-Universitaet Freiburg***, which appears in THE dataset as ***University of Freiburg***, and only a Google search, a local or an expert in universities could detect that this is actually the same university. All of the found matches could be seen below, where we finalize the list of matching universities:

In [205]:
paired.remove(('Universitat de Barcelona' ,'Autonomous University of Barcelona'))
paired.remove(('Vrije Universiteit Brussel (VUB)', 'Université Libre de Bruxelles'))

paired.append(('Ludwig-Maximilians-Universität München', 'LMU Munich'))
paired.append(('Albert-Ludwigs-Universitaet Freiburg', 'University of Freiburg'))
paired.append(('Lomonosov Moscow State University', 'Lomonosov Moscow State University'))
paired.append(('Universitat Autònoma de Barcelona', 'Autonomous University of Barcelona'))
paired.append(('Washington University in St. Louis', 'Washington University in St Louis'))

len(paired)

160

Finally, we merge the two datasets, keeping the information only for the universities that appear in the both datasets. For that, we will keep the name of the university as it appears in TU ranking. 

In [216]:
THE_names_to_TU_names = {key:value for value, key in paired} 
# dictionary which maps the universities names from THE ranking to the universities names from TU ranking, for all universities
# that appear in both rankings

THE_df = data_thedu.copy()

THE_df['name'] = THE_df['name'].apply(lambda x: THE_names_to_TU_names[x] if x in THE_names_to_TU_names else x)
# transforms the name of the universities that also appeat in TU ranking in their equivalent from TU
THE_df.rename(columns={'name':'title', 'rank':'rank_display', 'stats_number_students':'Total students', 
                      'stats_intl_students_ratio':'Ratio intl/total students', 
                      'stats_staff_student_ratio':'Ratio faculty/students',
                      'number_international_students_inferred':'International students',
                      'number_faculty_members_inferred':'Total faculty members'}, inplace=True)

merged_df = TU_df.merge(THE_df, on='title', how='inner', suffixes=('_TU', '_THE'))
merged_df


,country,rank_display_TU,region,score,title,Total faculty members_TU,Total students_TU,International faculty members,International students_TU,Ratio faculty/students_TU,Ratio intl/total students_TU,location,rank_display_THE,Total students_THE,Ratio intl/total students_THE,Ratio faculty/students_THE,International students_THE,Total faculty members_THE
0,United States,1,North America,100,Massachusetts Institute of Technology (MIT),2982.0,11067.0,1679.0,3717.0,0.269450,0.335863,United States,5,11177,0.34,0.114943,3800.18,1284.712644
1,United States,2,North America,98.7,Stanford University,4285.0,15878.0,2042.0,3611.0,0.269870,0.227422,United States,=3,15845,0.22,0.133333,3485.90,2112.666667
2,United States,3,North America,98.4,Harvard University,4350.0,22429.0,1311.0,5266.0,0.193945,0.234785,United States,6,20326,0.26,0.112360,5284.76,2283.820225
3,United States,4,North America,97.7,California Institute of Technology (Caltech),953.0,2255.0,350.0,647.0,0.422616,0.286918,United States,=3,2209,0.27,0.153846,596.43,339.846154
4,United Kingdom,5,Europe,95.6,University of Cambridge,5490.0,18770.0,2278.0,6699.0,0.292488,0.356899,United Kingdom,2,18389,0.35,0.091743,6436.15,1687.064220
5,United Kingdom,6,Europe,95.3,University of Oxford,6750.0,19720.0,2964.0,7353.0,0.342292,0.372870,United Kingdom,1,20409,0.38,0.089286,7755.42,1822.232143
6,United Kingdom,7,Europe,94.6,University College London,6345.0,31080.0,2554.0,14854.0,0.204151,0.477928,United Kingdom,16,30304,0.49,0.095238,14848.96,2886.095238
7,United Kingdom,8,Europe,93.7,Imperial College London,3930.0,16090.0,2071.0,8746.0,0.244251,0.543567,United Kingdom,8,15857,0.55,0.087719,8721.35,1390.964912
8,United States,9,North America,93.5,University of Chicago,2449.0,13557.0,635.0,3379.0,0.180645,0.249244,United States,9,13525,0.25,0.161290,3381.25,2181.451613
9,Switzerland,10,Europe,93.3,ETH Zurich - Swiss Federal Institute of Techno...,2477.0,19815.0,1886.0,7563.0,0.125006,0.381681,Switzerland,=10,19233,0.38,0.068493,7308.54,1317.328767
